In [1]:
import numpy as np
import pandas as pd
import re
from pandas import Series, DataFrame
gtes = "gtes10ene.xls"
goi = "goi10ene.xls"
gom = "gom10ene.xls"
gef = "gef10ene.xls"
cuentas=[7,10,11,12,16,17]
bdgtes = pd.read_excel('../data/'+gtes)
bdgoi = pd.read_excel('../data/'+goi)
bdgom = pd.read_excel('../data/'+gom)
bdgef = pd.read_excel('../data/'+gef)

bdBruto= pd.concat([bdgtes,bdgoi,bdgom,bdgef],ignore_index=True)
#bd.describe()


WARNING *** file size (522041) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (443859) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (3180501) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (4688657) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [2]:
#bdBruto.columns

In [3]:
bdReducida=bdBruto.loc[:,('cve_tipo_comprob','glosa_reng','cve_debe_haber','monto_mo',
           'cod_moneda', 'cod_movimiento', 'nom_movimiento',
           'monto_mn','glosa_comprob','nro_comprob','cod_mayor')]
#bdReducida

In [4]:
dict = {'cve_tipo_comprob':'tipo','glosa_reng':'glosaRenglon','cve_debe_haber':'dh','monto_mo':'mo',
           'cod_moneda':'moneda', 'cod_movimiento':'codMov', 'nom_movimiento':'nomMov',
           'monto_mn':'mn','glosa_comprob':'glosa','nro_comprob':'comprobante','cod_mayor':'mayor'}
bdReducida.rename(columns=dict,
          inplace=True)
bdReducida.shape

(6426, 11)

In [5]:
#bd['mayor'].unique()

#bdFiltro1=bdReducida[bdReducida['mayor'].isin([7,10,11,16,17])]
bdFiltro1=bdReducida[bdReducida['mayor'].isin(cuentas)]

In [6]:
#bdFiltro1.loc[:,("comprobante","mayor")]
bdFiltro1["comprobante"].unique()
#bdFiltro1.sort_values(by="comprobante", ascending=True)

array([1990252, 1990283, 1990300, 1990333, 1990403, 1023306,   28374,
       1023297, 1023298, 1023315,   21779, 1752256, 1752500, 1752501,
       1752255, 1023337, 1023304, 1752874, 1752033, 1023293, 1752380,
       1752382, 1023301, 1752502, 1752619, 1752621, 1752624, 1752627,
       1752630, 1752743, 1023343, 1752864, 1752866, 1752870, 1752872,
       1023350, 1023348, 1023345, 1023294], dtype=int64)

In [7]:
#bd[bd['comprobante'].isin(bdFiltro1["comprobante"])].sort_values(by="comprobante", ascending=True).tail(60)
bd=bdReducida[bdReducida["comprobante"].isin(bdFiltro1["comprobante"])].sort_values(by=["comprobante","dh"], ascending=True)
bd = bd.reset_index(drop=True)
#bd.head(60)

In [8]:
#list(bd.loc[55,["glosa"]])

In [9]:
#bd.to_excel("ux.xlsx")

In [10]:
bd=bd.drop_duplicates()
bd = bd.reset_index(drop=True)
#bd.head(60)

In [11]:
#bd["comprobante"].unique()

In [12]:
def llenarData(concepto,a,b):
    if concepto !="Neteo":    
        print('dentro ',pares["Comprobante"][a])
        data2.append({
                 "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":round(pares["MN"][a]/6.86,2),
                "Detalle":pares["GlosaDetalle"][b],"Glosa":pares["Glosa"][a]
                })
    else:
        data2.append({
                "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":0, 
                "Detalle":("±", round(pares["MN"][a]/6.86,2)),"Glosa":pares["Glosa"][a]
                })

def dv(concepto,suma):
    print("largo cuando dentra al debehaber: ",len(aux2))
    if len(aux2)==1:
        data2.append({''
                     "Tipo":aux2["tipo"][0],"Comprobante":aux2["comprobante"][0],"Concepto":concepto,
                    "Mayor1":aux2["mayor"][0],"Mayor2":"--","codMov":"--",
                    "Movimiento":"--","MontoUSD":round(suma,2),
                    "Detalle":aux2["glosaRenglon"][0],"Glosa":aux2["glosa"][0]
                    })
    else:
        data2.append({''
                     "Tipo":aux2["tipo"][0],"Comprobante":aux2["comprobante"][0],"Concepto":concepto,
                    "Mayor1":aux2["mayor"][0],"Mayor2":aux2["mayor"][1],"codMov":aux2["codMov"][1],
                    "Movimiento":aux2["nomMov"][1],"MontoUSD":round(suma,2),
                    "Detalle":aux2["glosaRenglon"][0],"Glosa":aux2["glosa"][0]
                    })

def debemenoshaber(hasta):
    sumaDebe=0
    sumaHaber=0
    for i in hasta:
        if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
            sumaDebe=sumaDebe+aux2["mn"][i]/6.86
        elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
            sumaHaber=sumaHaber+aux2["mn"][i]/6.86
    suma=sumaDebe-sumaHaber
    if suma>0:
        dv("Ingreso",suma)
    else:
        dv("Egreso",-suma)

In [13]:
aux1=bd["comprobante"].unique()
consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    print("Comprobante ",q)
    print(aux2["comprobante"][0])
    if aux2["tipo"][0]=='V' or aux2["tipo"][0]=='D': ####### los ajustes por arbitraje y revalorizacion ######
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
        sumaDebe=0
        sumaHaber=0
        for i in aux2.index:
            if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
                sumaDebe=sumaDebe+aux2["mn"][i]/6.86
            elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
                sumaHaber=sumaHaber+aux2["mn"][i]/6.86
        suma=sumaDebe-sumaHaber
        if suma>0:
            dv("Ingreso",suma)
            #unRegistro=pd.DataFrame(data2,columns=columnas)
            #consolidado=consolidado.append(unRegistro, ignore_index=True)
        else:
            dv("Egreso",-suma)
            #unRegistro=pd.DataFrame(data2,columns=columnas)
            #consolidado=consolidado.append(unRegistro, ignore_index=True)
        unRegistro=pd.DataFrame(data2,columns=columnas)
        consolidado=consolidado.append(unRegistro, ignore_index=True)
    else: # estos son los casos normales digamos ##############################################################
        #print ("menos Tipo D o V")
        print("Hay duplicado: ",aux2.duplicated(subset=['mn']).any())
        
        if aux2.duplicated(subset=['mn']).any()==True:  ## aqui sacamos los repetidos en la MN
            aux3=aux2.duplicated(subset=['mn'],keep=False)
            aux4=aux3.index[aux3==True].tolist()
            print('aux4')
            print(aux4)
            for x in range(len(aux4)):

                data1.append({
                    "Tipo":aux2["tipo"][aux4[x]],"Comprobante":aux2["comprobante"][aux4[x]],"Concepto":aux2["dh"][aux4[x]],
                    "Mayor":aux2["mayor"][aux4[x]],"codMov":aux2["codMov"][aux4[x]],"Movimiento":aux2["nomMov"][aux4[x]],
                    "MN":aux2["mn"][aux4[x]],"Glosa":aux2["glosa"][aux4[x]],"GlosaDetalle":aux2["glosaRenglon"][aux4[x]],
                            })
                pares=pd.DataFrame(data1,columns=columnas)

        else:   # Aqui anotamos todos los que no son repetidos
            print('aux2')
            aux2= aux2.sort_values(by='mayor') #, key=cuentas)
            aux2=aux2.reset_index(drop=True)
            print(aux2)
            
            for x in range(len(aux2)):
                #print(aux[x],"=== ",aux2["mn"][aux4[x]])
                data1.append({
                    "Tipo":aux2["tipo"][x],"Comprobante":aux2["comprobante"][x],"Concepto":aux2["dh"][x],
                    "Mayor":aux2["mayor"][x],"codMov":aux2["codMov"][x],"Movimiento":aux2["nomMov"][x],
                    "MN":aux2["mn"][x],"Glosa":aux2["glosa"][x],"GlosaDetalle":aux2["glosaRenglon"][x],
                            })
                pares=pd.DataFrame(data1,columns=columnas)

            #print(aux2['mayor'][i],' ... ',aux2["comprobante"][i])
            #print('pares ',pares["Comprobante"][0])
        print(pares)
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes

        if len(aux2)==2: ### los pares. Este sería el caso normal
            print("largo de aux2: ",len(aux2))
            if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
                #print("Posible Neteo")
                llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
                unRegistro=pd.DataFrame(data2,columns=columnas)
            else:     
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
                    print('caso 1')
                    llenarData("Ingreso",0,1)        
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
                    print('caso 2')
                    llenarData("Egreso",0,1)                   
                if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
                    print('caso 3')
                    llenarData("Ingreso",1,0)                    
                if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
                    print('caso 4')
                    llenarData("Egreso",1,0)
        else:
            if len(aux2)==1:  ### los sueltitos (se supone que los otros movimientos estan en otras areas, ej. gadm)              
                print("entramos con 1")
                if pares["Concepto"][0]=="D":
                    print('caso 5')
                    llenarData("Ingreso",0,0)
                else:
                    print('caso 6')
                    llenarData("Egreso",0,0)
            else:             # Aqui es cuando no hay duplicados y no son pares ni sueltitos
                print("mas de 2")
                ###
                data2=[]
                columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
                sumaDebe=0
                sumaHaber=0
                for i in range(len(aux2)):
                    if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
                        sumaDebe=sumaDebe+aux2["mn"][i]/6.86
                    elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
                        sumaHaber=sumaHaber+aux2["mn"][i]/6.86
                suma=sumaDebe-sumaHaber
                if suma>0:
                    dv("Ingreso",suma)
                    #unRegistro=pd.DataFrame(data2,columns=columnas)
                    #consolidado=consolidado.append(unRegistro, ignore_index=True)
                else:
                    dv("Egreso",-suma)
                    #unRegistro=pd.DataFrame(data2,columns=columnas)
                    #consolidado=consolidado.append(unRegistro, ignore_index=True)
                ####

        unRegistro=pd.DataFrame(data2,columns=columnas)
        consolidado=consolidado.append(unRegistro, ignore_index=True)  

consolidado
#consolidado.to_excel("borrar1.xlsx")

Comprobante  0
21779
largo cuando dentra al debehaber:  24
Comprobante  1
28374
largo cuando dentra al debehaber:  19
Comprobante  2
1023293
Hay duplicado:  True
aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1023293        D    498    8304   
1    S      1023293        H     10    5896   

                                          Movimiento         MN  \
0  ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...  3301032.0   
1  NEW YORK - CTA. 3544020682001                 ...  3301032.0   

                                               Glosa  \
0  'DEVOLUCION POR TRANSFERENCIA RECIBIDA DEL EXT...   
1  'DEVOLUCION POR TRANSFERENCIA RECIBIDA DEL EXT...   

                 GlosaDetalle  
0        (CC:H-0000048-2022)   
1  SEGÚN GLOSA DE REFERENCIA.  
largo de aux2:  2
caso 4
dentro  1023293
Comprobante  3
1023294
Hay duplicado:  False
aux2
  tipo          glosaRenglon dh       mo  moneda  codMov  \
0    S                   NaN  H  7208.95      34    5896   
1    S

aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1752382        D     10    5896   
1    G      1752382        H    412     818   

                                          Movimiento         MN  \
0  NEW YORK - CTA. 3544020682001                 ...  382633.92   
1  CTAS. CTES. FISC. CORRESP. -BCO.UNION         ...  382633.92   

                                               Glosa  \
0  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.251 ...   
1  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.251 ...   

                                        GlosaDetalle  
0                                                NaN  
1  CTA. 10000010728414 EMPRESA MINERA COLQUIRI - ...  
largo de aux2:  2
caso 1
dentro  1752382
Comprobante  20
1752500
Hay duplicado:  False
aux2
  tipo                                       glosaRenglon dh          mo  \
0    G                                                NaN  H   695295.89   
1    G  CTA. 10000009590155 MMP1217 GAMLP-ADMINISTRACI...  D  4839

aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1990300        D      7    3491   
1    O      1990300        H    414    7245   

                                          Movimiento         MN  \
0  DOLARES AMERICANOS EN TESORO PROPIO           ...  110740.98   
1  DEPOSITOS POR APORTES FPA - LEY 365           ...  110740.98   

                                               Glosa  GlosaDetalle  
0  DEPOSITO DE EFECTIVO, DEPOSITANTE: SEGUROS Y R...           NaN  
1  DEPOSITO DE EFECTIVO, DEPOSITANTE: SEGUROS Y R...           NaN  
largo de aux2:  2
caso 1
dentro  1990300
Comprobante  37
1990333
Hay duplicado:  True
aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1990333        D    403    3910   
1    O      1990333        H      7    3491   

                                          Movimiento          MN  \
0  BCO.UNION-CUENTA CORRIENTE Y DE ENCAJE        ...  24010000.0   
1  DOLARES AMERICANOS EN TESORO PROPIO           ...  2401000

,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,D,21779,Egreso,498,10,7080,BANK OF AMERICA MERRILL LYNCH MIAMI 1901-3-282...,0.00,Saldo ME = 0.00 ;Bs/Mo: 6.86000000000 ;Saldo B...,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...
1,V,28374,Ingreso,306,17,6296,INTERESES DEVENGADOS ...,4086550.56,Saldo Ant.= -36070.00 ;Bs: 9.59151480000 = $us...,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...
2,S,1023293,Egreso,10,498,8304,ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...,481200.00,(CC:H-0000048-2022),'DEVOLUCION POR TRANSFERENCIA RECIBIDA DEL EXT...
3,S,1023294,Egreso,10,198,3641,DEUDORES DIVERSOS-OPERACIONES CAMBIARIAS ...,7208.95,NaN,||PAGO COMISIONES Y GASTOS BANCARIOS AL STANDA...
4,S,1023297,Neteo,10,17,6296,INTERESES DEVENGADOS ...,0.00,"(±, 735702.39)","||PAGO DE CUPON DE UN TITULO RAIFFS 0,42 01/10..."
5,S,1023298,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,6130.85,NaN,||COMPLEMENTO A COMPROBANTE N° 1023297 DE LA F...
6,S,1023301,Ingreso,10,414,7866,EFECTIVO GARANTIA FONDO CPVIS ...,248125.00,NaN,||TRANSF. DE FONDOS DEL EXTERIOR SEGUN SWIFT N...
7,S,1023304,Ingreso,16,499,7355,VALORACION TITULOS DE LAS RESERVAS INTERNACION...,38.90,NaN,||REVALORACIÓN DE TITULOS A PRECIOS DE MERCA...
8,S,1023306,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,11718.30,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...
9,S,1023315,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,10138.94,NaN,||INTERESES DEVENGADOS AL 10 DE ENERO DE 2...


In [16]:
aux1=bd["comprobante"].unique()
consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    print("Comprobante ",q)
    print(aux2["comprobante"][0])
    if aux2["tipo"][0]=='V' or aux2["tipo"][0]=='D': ####### los ajustes por arbitraje y revalorizacion ######
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
        ########## RRRRRRRRRRRR
        debemenoshaber(aux2.index)
        ########## RRRRRRRRRRRR   
        unRegistro=pd.DataFrame(data2,columns=columnas)
        consolidado=consolidado.append(unRegistro, ignore_index=True)
    else: # estos son los casos normales digamos ##############################################################
        
        print("Hay duplicado: ",aux2.duplicated(subset=['mn']).any())
        
        if aux2.duplicated(subset=['mn']).any()==True:  ## aqui sacamos los repetidos en la MN
            aux3=aux2.duplicated(subset=['mn'],keep=False)
            aux4=aux3.index[aux3==True].tolist()
            print('aux4')
            print(aux4)
            if len(aux4)==2:
                for x in range(len(aux4)):

                    data1.append({
                        "Tipo":aux2["tipo"][aux4[x]],"Comprobante":aux2["comprobante"][aux4[x]],"Concepto":aux2["dh"][aux4[x]],
                        "Mayor":aux2["mayor"][aux4[x]],"codMov":aux2["codMov"][aux4[x]],"Movimiento":aux2["nomMov"][aux4[x]],
                        "MN":aux2["mn"][aux4[x]],"Glosa":aux2["glosa"][aux4[x]],"GlosaDetalle":aux2["glosaRenglon"][aux4[x]],
                                })
                    pares=pd.DataFrame(data1,columns=columnas)
                #################### 1
                print("largo de aux2: ",len(aux2))
                print(pares)
                data2=[]
                columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
                    #print("Posible Neteo")
                    llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
                    unRegistro=pd.DataFrame(data2,columns=columnas)
                else:     
                    if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
                        print('caso 1')
                        llenarData("Ingreso",0,1)        
                    if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
                        print('caso 2')
                        llenarData("Egreso",0,1)                   
                    if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
                        print('caso 3')
                        llenarData("Ingreso",1,0)                    
                    if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
                        print('caso 4')
                        llenarData("Egreso",1,0)
            else:
                
                #@#@#@#@#@#@#
                data2=[]
                columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
               
                ####BBBBBBBBBBBBBBBBBBB
                debemenoshaber(range(len(aux2)))
                ####BBBBBBBBBBBBBBBBBBB
                #@#@#@#@#@#@#
            #################### 1

        else:   # Aqui anotamos todos los que no son repetidos
            print('aux2')
            aux2= aux2.sort_values(by='mayor') #, key=cuentas)
            aux2=aux2.reset_index(drop=True)
            print(aux2)
            
            for x in range(len(aux2)):
                #print(aux[x],"=== ",aux2["mn"][aux4[x]])
                data1.append({
                    "Tipo":aux2["tipo"][x],"Comprobante":aux2["comprobante"][x],"Concepto":aux2["dh"][x],
                    "Mayor":aux2["mayor"][x],"codMov":aux2["codMov"][x],"Movimiento":aux2["nomMov"][x],
                    "MN":aux2["mn"][x],"Glosa":aux2["glosa"][x],"GlosaDetalle":aux2["glosaRenglon"][x],
                            })
                pares=pd.DataFrame(data1,columns=columnas)
            ################### 2
            print(pares)
            data2=[]
            columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
            
            if len(aux2)==1:  ### los sueltitos (se supone que los otros movimientos estan en otras areas, ej. gadm)              
                print("entramos con 1")
                if pares["Concepto"][0]=="D":
                    print('caso 5')
                    llenarData("Ingreso",0,0)
                else:
                    print('caso 6')
                    llenarData("Egreso",0,0)
            else:             # Aqui es cuando no hay duplicados y no son pares ni sueltitos
                print("mas de 2")
                
                
                data2=[]
                columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
                ######### HHHHHHH
               # if len(aux2)==3 and aux2["mayor"][0] in cuentas and aux2["mayor"][1] in cuentas and aux2["mayor"][2]==680:
               #     llenarData("Neteo",0,1)
               # else:
                debemenoshaber(range(len(aux2)))
               
                ######### HHHHHHH 
            ################### 2
            #print(aux2['mayor'][i],' ... ',aux2["comprobante"][i])
            #print('pares ',pares["Comprobante"][0])
            
        ###$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        ##### $$$$$$$$$$$$$$$
        unRegistro=pd.DataFrame(data2,columns=columnas)
        consolidado=consolidado.append(unRegistro, ignore_index=True)  

consolidado

#consolidado.to_excel("borrar10ene.xlsx")

Comprobante  0
21779
largo cuando dentra al debehaber:  24
Comprobante  1
28374
largo cuando dentra al debehaber:  19
Comprobante  2
1023293
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1023293        D    498    8304   
1    S      1023293        H     10    5896   

                                          Movimiento         MN  \
0  ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...  3301032.0   
1  NEW YORK - CTA. 3544020682001                 ...  3301032.0   

                                               Glosa  \
0  'DEVOLUCION POR TRANSFERENCIA RECIBIDA DEL EXT...   
1  'DEVOLUCION POR TRANSFERENCIA RECIBIDA DEL EXT...   

                 GlosaDetalle  
0        (CC:H-0000048-2022)   
1  SEGÚN GLOSA DE REFERENCIA.  
caso 4
dentro  1023293
Comprobante  3
1023294
Hay duplicado:  False
aux2
  tipo          glosaRenglon dh       mo  moneda  codMov  \
0    S                   NaN  H  7208.95      34    5896   
1    S

  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1752382        D     10    5896   
1    G      1752382        H    412     818   

                                          Movimiento         MN  \
0  NEW YORK - CTA. 3544020682001                 ...  382633.92   
1  CTAS. CTES. FISC. CORRESP. -BCO.UNION         ...  382633.92   

                                               Glosa  \
0  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.251 ...   
1  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.251 ...   

                                        GlosaDetalle  
0                                                NaN  
1  CTA. 10000010728414 EMPRESA MINERA COLQUIRI - ...  
caso 1
dentro  1752382
Comprobante  20
1752500
Hay duplicado:  False
aux2
  tipo                                       glosaRenglon dh          mo  \
0    G                                                NaN  H   695295.89   
1    G  CTA. 10000009590155 MMP1217 GAMLP-ADMINISTRACI...  D  4839259.39   
2    G  CTA. 1000000

1990283
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1990283        D    403    3914   
1    O      1990283        H      7    3491   

                                          Movimiento          MN  \
0  BISA-CUENTA CORRIENTE Y DE ENCAJE             ...  89180000.0   
1  DOLARES AMERICANOS EN TESORO PROPIO           ...  89180000.0   

                                               Glosa  GlosaDetalle  
0  PAGO CHEQUE: D-20-98911     , BENEFICIARIO: EM...           NaN  
1  PAGO CHEQUE: D-20-98911     , BENEFICIARIO: EM...           NaN  
caso 4
dentro  1990283
Comprobante  36
1990300
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1990300        D      7    3491   
1    O      1990300        H    414    7245   

                                          Movimiento         MN  \
0  DOLARES AMERICANOS EN TESORO PROPIO           ...  110740.98   
1  DEPOSITOS 

,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,D,21779,Egreso,498,10,7080,BANK OF AMERICA MERRILL LYNCH MIAMI 1901-3-282...,0.00,Saldo ME = 0.00 ;Bs/Mo: 6.86000000000 ;Saldo B...,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...
1,V,28374,Ingreso,306,17,6296,INTERESES DEVENGADOS ...,4086550.56,Saldo Ant.= -36070.00 ;Bs: 9.59151480000 = $us...,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...
2,S,1023293,Egreso,10,498,8304,ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...,481200.00,(CC:H-0000048-2022),'DEVOLUCION POR TRANSFERENCIA RECIBIDA DEL EXT...
3,S,1023294,Egreso,10,198,3641,DEUDORES DIVERSOS-OPERACIONES CAMBIARIAS ...,7208.95,NaN,||PAGO COMISIONES Y GASTOS BANCARIOS AL STANDA...
4,S,1023297,Neteo,10,17,6296,INTERESES DEVENGADOS ...,0.00,"(±, 735702.39)","||PAGO DE CUPON DE UN TITULO RAIFFS 0,42 01/10..."
5,S,1023298,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,6130.85,NaN,||COMPLEMENTO A COMPROBANTE N° 1023297 DE LA F...
6,S,1023301,Ingreso,10,414,7866,EFECTIVO GARANTIA FONDO CPVIS ...,248125.00,NaN,||TRANSF. DE FONDOS DEL EXTERIOR SEGUN SWIFT N...
7,S,1023304,Ingreso,16,499,7355,VALORACION TITULOS DE LAS RESERVAS INTERNACION...,38.90,NaN,||REVALORACIÓN DE TITULOS A PRECIOS DE MERCA...
8,S,1023306,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,11718.30,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...
9,S,1023315,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,10138.94,NaN,||INTERESES DEVENGADOS AL 10 DE ENERO DE 2...


In [15]:
texto1 = "PAGO A EXIMBANK CHINA POPUL PRÉSTAMO PBC 2015 (08) 350 VCTO. 21-01-2022 POR CUENTA DE TGN , NTI. 015626 VALOR 21-01-2022 CAPITAL USD 27.355.555,56 COMISIONES USD 260.358,75"
texto2= "PAGO A AFD PRÉSTAMO CBO-1009-01-J VCTO. 21-01-2022 POR CUENTA DE ENDE , VALOR 21-01-2022  INTERESES EUR 168.666,67"

#capital = re.findall(" CAPITAL (.*)(?= [IC])" , texto1)
capital = re.findall(" CAPITAL (.*)(?= [INTERESES][COMISIONES])" , texto1)
intereses = re.findall(" INTERESES (.*)(?=[C\s])" , texto1)
comisiones = re.findall(" COMISIONES (.*)" , texto1)

#t= pd.DataFrame(search)
#t
print(capital, intereses, comisiones)

[] [] ['USD 260.358,75']
